# Further GSEA enrichment analysis for group comparisons

* By: Hunter Giles
* Modifications by: Arthur Feltrin
* Modifications 07/2023 by Hunter Giles
* Modifications 01/05/2025 by Alan Lorenzetti

In [1]:
# Exploring the use of group size parameter limits for the GO term analysis
# (https://stephenturner.github.io/deseq-to-fgsea/) does not use size limits, but examines only hallmark GO terms
# (https://github.com/ctlab/fgsea) uses a range from n=15 to n=500
# We originally ran the analysis with n >= 15

# Load Packages

In [2]:
suppressMessages({
    library(tidyverse)
    library(readr)
    library(fgsea)
    library(ggpubr)
    library(data.table)
    library(grid)
    })

dir.create('../_m', showWarnings = TRUE, recursive = TRUE)
setwd('../_m')

save_plot <- function(p, fn, w, h){
    for(ext in c(".pdf", ".png")){
        ggsave(filename=paste0(fn,ext), plot=p, width=w, height=h)
    }
}

Warning message in dir.create("../_m", showWarnings = TRUE, recursive = TRUE):
“'../_m' already exists”


In [3]:
getwd()

[1] "/dcs05/lieber/apaquola/users/alorenzetti/xdp_striatal_organoids_RNA-seq/analysis_speaqeasy/deseq2_multiplemodels_genes_no_lfcShrink_filter_low/gsea/_m"

In [4]:
# Load Dataframes

We are only examining four group comparisons from the models that were run:
XDP vs control (dataset 1) should be males only: Mature_organoids_male_nodeltaSVAcomparison_deseq2   
XDP vs. dSVA (dataset 2) should be males only: Mature_organoids_male_DELTASVAcomparison_deseq2   
dSVA vs. control (dataset 3): Mature_organoids_male_ControlxDELTASVAfullmodel_deseq2   
Patient vs. control (dataset 4, current one in paper) should contain males and females:    Mature_organoids_all_comparison_design1_deseq2

In [5]:
# ****Edit paths in final file position****
comparisons <- c("XDP_v_Control", "XDP_v_dSVA", "dSVA_v_Control", "Patient_v_Control")
path_to_files <- c("../../_m/Mature_organoids_male__nodeltaSVA_comparison_deseq2/gene_deseq2_results.tsv",
                   "../../_m/Mature_organoids_male__DELTASVA_comparison_deseq2/gene_deseq2_results.tsv",
                   "../../_m/Mature_organoids_male__ControlxDELTASVA_fullmodel_deseq2/gene_deseq2_results.tsv",
                   "../../_m/Mature_organoids_all_comparison_design1_deseq2/gene_deseq2_results.tsv")

names(path_to_files) <- comparisons
print(path_to_files)

                                                                              XDP_v_Control 
     "../../_m/Mature_organoids_male__nodeltaSVA_comparison_deseq2/gene_deseq2_results.tsv" 
                                                                                 XDP_v_dSVA 
       "../../_m/Mature_organoids_male__DELTASVA_comparison_deseq2/gene_deseq2_results.tsv" 
                                                                             dSVA_v_Control 
"../../_m/Mature_organoids_male__ControlxDELTASVA_fullmodel_deseq2/gene_deseq2_results.tsv" 
                                                                          Patient_v_Control 
          "../../_m/Mature_organoids_all_comparison_design1_deseq2/gene_deseq2_results.tsv" 


In [6]:
getwd()

[1] "/dcs05/lieber/apaquola/users/alorenzetti/xdp_striatal_organoids_RNA-seq/analysis_speaqeasy/deseq2_multiplemodels_genes_no_lfcShrink_filter_low/gsea/_m"

In [7]:
#Load Gene Set List For Analysis

##### Set relative path in final file position #######

go_BP_MF_pathways <- gmtPathways("../_h/msigdb_v2023.1.Hs_GMTs/c5.go.bp.v2023.1.Hs.symbols.gmt")
# length(go_BP_MF_pathways)
# Should match 7751 gene sets
go_BP_MF_pathways <- c(go_BP_MF_pathways, gmtPathways("../_h/msigdb_v2023.1.Hs_GMTs/c5.go.mf.v2023.1.Hs.symbols.gmt"))
# length(go_BP_MF_pathways)
# Should match 7751 + 1772 = 9523 gene sets

# Create GO Enrichment Sets


In [8]:
# Cycle through each DESeq2 results table, creating an enrichment set for each

In [9]:
for (i in seq_along(path_to_files)){
    suppressMessages(data <- read_tsv(path_to_files[i]))
    
    #For duplicated genesymbols, get always the one with the largest basemean value
    #Arrange all results by baseMean expression
    #Take top 10,000 most expressed genes
    #Rank by Wald-Statistic - Patient vs healthy
    
#alternative1
    tenK_data <- data %>% 
        group_by(Symbol) %>%
        arrange(Symbol, desc(baseMean.x)) %>%
        filter(row_number()==1) %>%
        arrange(desc(baseMean.x)) %>%
        slice(1:10000) %>%
        select(Symbol, stat) %>%
        arrange(desc(stat))
    
#alternative2
    # tenK_data <- data %>% 
    #     arrange(Symbol, desc(baseMean.x)) %>%
    #     distinct(Symbol, .keep_all = TRUE) %>%
    #     arrange(desc(baseMean.x)
    #     slice(1:10000) %>%
    #     select(Symbol, stat) %>%
    #     arrange(desc(stat))
    
    #Produce vector of gene ranks
    gene_ranks <- deframe(tenK_data)
    
    #Run fgsea using minimum gene set size of 15 and 10000 permutations for Enrichment Score calculations
    res_go <- fgsea(pathways = go_BP_MF_pathways, gene_ranks, minSize = 15, maxSize = 500, nPermSimple = 10000) %>% as_tibble()
    
    #Pull top ten and bottom fifty results by Normalized Enrichment Score, merge to a single tibble and create vector of pathways - GO results
    top_res_go <- res_go %>%
        arrange(desc(NES)) %>%
        slice_head(n=10)
    bottom_res_go <- res_go %>%
        arrange(desc(NES)) %>%
        slice_tail(n=10)
    to_plot_go <- bind_rows(top_res_go, bottom_res_go)
    #print(to_plot)
    to_plot_pathways_go <- to_plot_go %>% pull(pathway)
    #print(to_plot_pathways)

    #Set Color Preferences
    colrs <- setNames(c("darkgrey", "firebrick2"),
                 c("FALSE", "TRUE"))
    #print(colrs)
    
    #Make the DotPlot for GO pathways
    dot_plot_go <- ggplot(to_plot_go, aes(reorder(pathway, NES), NES)) +
        geom_point( aes(fill = padj < 0.05, size = size), shape=21) +
        scale_fill_manual(values = colrs ) +
        scale_size_continuous(range = c(2,10)) +
        geom_hline(yintercept = 0) +
        coord_flip() +
        labs(x = "Pathway", y = "Normalized Enrichment Score",
           title = "Enriched GO Pathways") + 
        theme_pubr() +
        theme(legend.position = "bottom", plot.margin = unit(c(0.15,0.8,0.15,0.15), "cm"))
    
    pdf(file = paste0(comparisons[i], '_GOresults.pdf'), width = 14, height = 8)
    
    print(plotGseaTable(pathways=go_BP_MF_pathways[to_plot_pathways_go],
                  stats = gene_ranks, 
                  fgseaRes = res_go, 
                  gseaParam=1, colwidths = c(4, 2, 0.5, 0.5, 0.5)
                 ))
    print(dot_plot_go)
    dev.off()
    
    fwrite(res_go %>% arrange(desc(NES)), paste0(comparisons[i],'_GOresults.tsv'),sep='\t',quote=F,row.names=F)
                    
   # png(file = paste0(tags[i], '.png'), width = 800, height = 800)
  #  table_plot
  #  dot_plot
   # dot_plot_bottom_legend
    #dev.off()    
    
}

In [10]:
sessionInfo()

R version 4.4.2 (2024-10-31)
Platform: x86_64-pc-linux-gnu
Running under: Arch Linux

Matrix products: default
BLAS:   /usr/lib/libblas.so.3.12.0 
LAPACK: /usr/lib/liblapack.so.3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: EST
tzcode source: system (glibc)

attached base packages:
[1] grid      stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] data.table_1.16.4 ggpubr_0.6.0      fgsea_1.30.0      lubridate_1.9.4  
 [5] forcats_1.0.0     stringr_1.5.1     dplyr_1.1.4       purrr_1.0.2      
 [9] readr_2.1.5       tidyr_1.3.1       tibble_3.2.1      ggplot2_3.5.1    
[13] tidyverse_2.0.0  

loaded via a 